In [1]:
import torch
import torch.nn as nn
from torchvision import datasets, transforms
from torch.autograd import Variable
from tqdm import tqdm
import numpy as np

In [2]:
import torch.utils.data as Data
    
BATCH_SIZE = 100

train_dataset = datasets.MNIST(root='./data/',train=True,transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))
                       ]))
test_dataset = datasets.MNIST(root='./data/',train=False,transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))
                       ]))

train_loader = Data.DataLoader(dataset=train_dataset,batch_size=BATCH_SIZE,shuffle=True)
test_loader = Data.DataLoader(dataset=test_dataset,batch_size=BATCH_SIZE,shuffle=False)

In [3]:
def Binarize(x):
    return torch.where(x >= 0, torch.full_like(x, 1), torch.full_like(x, -1))

    
class BinarizeLinear(nn.Linear):

    def __init__(self, *kargs, **kwargs):
        super(BinarizeLinear, self).__init__(*kargs, **kwargs)

    def forward(self, input):

        if input.size(1) != 784:
            input.data=Binarize(input.data)
        if not hasattr(self.weight,'org'):
            self.weight.org=self.weight.data.clone()
        self.weight.data=Binarize(self.weight.org)
        out = nn.functional.linear(input, self.weight)
        if not self.bias is None:
            self.bias.org=self.bias.data.clone()
            out += self.bias.view(1, -1).expand_as(out)

        return out

In [4]:
class BinaryNet(nn.Module):
    def __init__(self):
        super(BinaryNet, self).__init__()
        self.infl_ratio=4
        self.layers=nn.Sequential(
            
            BinarizeLinear(784, 1024*self.infl_ratio),
         #   nn.Linear(784, 1024*self.infl_ratio),
            nn.BatchNorm1d(1024*self.infl_ratio),
            nn.Hardtanh(),
            
            BinarizeLinear(1024*self.infl_ratio, 1024*self.infl_ratio),
      #      nn.Linear(1024*self.infl_ratio,1024*self.infl_ratio),
            nn.BatchNorm1d(1024*self.infl_ratio),
            nn.Hardtanh(),
            
            BinarizeLinear(1024*self.infl_ratio, 1024*self.infl_ratio),
      #      nn.Linear(1024*self.infl_ratio,1024*self.infl_ratio),
            nn.BatchNorm1d(1024*self.infl_ratio),
            nn.Hardtanh(),
            
            nn.Linear(1024*self.infl_ratio,10),
            nn.BatchNorm1d(10)
        )
    def forward(self, x):
        x = x.view(-1, 784)
        x = self.layers(x)
        return x
    

In [5]:
binaryModel = BinaryNet()
binaryModel.cuda()

BinaryNet(
  (layers): Sequential(
    (0): BinarizeLinear(in_features=784, out_features=4096, bias=True)
    (1): BatchNorm1d(4096, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): Hardtanh(min_val=-1.0, max_val=1.0)
    (3): BinarizeLinear(in_features=4096, out_features=4096, bias=True)
    (4): BatchNorm1d(4096, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): Hardtanh(min_val=-1.0, max_val=1.0)
    (6): BinarizeLinear(in_features=4096, out_features=4096, bias=True)
    (7): BatchNorm1d(4096, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): Hardtanh(min_val=-1.0, max_val=1.0)
    (9): Linear(in_features=4096, out_features=10, bias=True)
    (10): BatchNorm1d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
)

In [6]:
class HingeLoss(torch.nn.Module):
    def __init__(self):
        super(HingeLoss, self).__init__()

    def forward(self, output, target):
        hinge_loss = 1 - torch.mul(output, target)
        hinge_loss[hinge_loss < 0] = 0
        return torch.mean(hinge_loss)

In [7]:
import torch.optim as optim
criterion = HingeLoss()
optimizer = optim.Adam(binaryModel.parameters(), lr=1e-3)

In [14]:
def train(epoch):
    binaryModel.train()
    prbar = tqdm(total=len(train_loader))
    prbar.set_description("training epoch"+str(epoch))
    for data, target in train_loader:
        onehot = (-1*torch.ones(BATCH_SIZE, 10)).scatter_(1, target.unsqueeze(-1), 1) # one-hot
        data, onehot = data.cuda(), onehot.cuda()
        data, onehot = Variable(data), Variable(onehot)
        optimizer.zero_grad()
        output = binaryModel(data)
        loss = criterion(output, onehot)


        optimizer.zero_grad()
        loss.backward()
        for p in list(binaryModel.parameters()):
            if hasattr(p,'org'):
                p.data.copy_(p.org)
        optimizer.step()
        for p in list(binaryModel.parameters()):
            if hasattr(p,'org'):
                p.org.copy_(p.data.clamp_(-1,1))
        prbar.update(1)
        prbar.set_postfix(loss=loss.item())
    prbar.close()
   # print('Train Epoch: {} \tLoss: {:.4f}'.format(epoch, loss.item()))
    
def test():
    binaryModel.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            onehot = (-1*torch.ones(BATCH_SIZE, 10)).scatter_(1, target.unsqueeze(-1), 1) # one-hot
            target = Variable(target.cuda())
            data, onehot = data.cuda(), onehot.cuda()
            data, onehot = Variable(data), Variable(onehot)
            output = binaryModel(data)
            test_loss += criterion(output, onehot).item() # sum up batch loss
            pred = output.data.max(1, keepdim=True)[1] # get the index of the max log-probability
            correct += pred.eq(target.data.view_as(pred)).cpu().sum()

    test_loss /= len(test_loader.dataset)
    print('Test :  Accuracy: {}/{} ({:.2f}%)\n'.format(
        correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [ ]:
for epoch in range(1, 10):
    train(epoch)
    test()